In [11]:
import torch
import numpy as np

In [12]:
torch.__version__

'1.13.1'

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra

from pyroomacoustics.directivities import (
    DirectivityPattern,
    DirectionVector,
    CardioidFamily,
)

In [14]:
def room_properties(absorption, size):
    if absorption=='high':
        absor = 0.7
    elif absorption=='medium':
        absor = 0.3
    elif absorption=='low':
        absor = 0.1
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    
    if size=='large':
        size_coef = 5.0
    elif size=='medium':
        size_coef = 2.5
    elif size=='small':
        size_coef = 1.0
    else:
        raise ValueError("The absorption parameter can only take values ['low', 'medium', 'high']")
    return absor, size_coef

In [15]:
def clip(signal, high, low):
    """Clip a signal from above at high and from below at low."""
    s = signal.copy()

    s[np.where(s > high)] = high
    s[np.where(s < low)] = low

    return s

def normalize(signal, bits=None):
    """
    normalize to be in a given range. The default is to normalize the maximum
    amplitude to be one. An optional argument allows to normalize the signal
    to be within the range of a given signed integer representation of bits.
    """

    s = signal.copy()

    s /= np.abs(s).max()

    # if one wants to scale for bits allocated
    if bits is not None:
        s *= 2 ** (bits - 1) - 1
        s = clip(s, 2 ** (bits - 1) - 1, -(2 ** (bits - 1)))

    return s
    
def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):
    """
    Save all the signals to wav files.

    Parameters
    ----------
    filename: str
        the name of the file
    mono: bool, optional
        if true, records only the center channel floor(M / 2) (default
        `False`)
    norm: bool, optional
        if true, normalize the signal to fit in the dynamic range (default
        `False`)
    bitdepth: int, optional
        the format of output samples [np.int8/16/32/64 or np.float
        (default)]
    """
    from scipy.io import wavfile

    signal = signals.T  # each column is a channel

    float_types = [float, np.float, np.float32, np.float64]

    if bitdepth in float_types:
        bits = None
    elif bitdepth is np.int8:
        bits = 8
    elif bitdepth is np.int16:
        bits = 16
    elif bitdepth is np.int32:
        bits = 32
    elif bitdepth is np.int64:
        bits = 64
    else:
        raise NameError("No such type.")

    if norm:
        signal = normalize(signal, bits=bits)

    signal = np.array(signal, dtype=bitdepth)

    wavfile.write(filename, fs, signal)

/tmp/ipykernel_42082/3403348603.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def to_wav(filename, fs, signals, norm=False, bitdepth=np.float):


In [6]:
import glob

In [7]:
a = glob.glob('/nvme/zhiyong/CN-Celeb/data/*')

In [8]:
c = []
for i in a:
    b = glob.glob(i+'/*')
    if b[0].split('/')[-1].split('-')[0] == 'interview':
        c.append(i)

In [9]:
a = c

# G7

In [10]:
import glob

In [11]:
b = a[0:60]

In [12]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [13]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-056.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-061.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-072.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-020.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-043.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-080.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-065.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-01-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-014.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-059.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-066.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-02-022.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-074.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-027.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00644/interview-03-004.wav',
 '/nvme/zhiyong/CN-Celeb

In [14]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('high', 'small')
    room = pra.ShoeBox(
        p=size_coef*np.array([4, 3, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[3.5,2.5,1.])

    room.simulate()
    
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [15]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G7/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G7.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G8

In [16]:
import glob

In [17]:
b = a[60:120]

In [18]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [19]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00082/interview-06-003.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/entertainment-02-011.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-07-016.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/entertainment-01-015.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-07-034.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-05-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-06-009.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-03-020.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-01-005.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-03-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/entertainment-01-025.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-08-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/entertainment-01-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-01-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-08-013.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00082/interview-03-014.wav',
 '/nvme/

In [20]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('low', 'large')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[17.,12.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [21]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G8/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G8.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G9

In [22]:
import glob

In [23]:
b = a[120:180]

In [24]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [25]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00254/interview-05-029.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-06-003.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-05-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-06-009.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-01-005.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-03-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-05-012.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-01-042.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-01-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-03-014.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-03-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-03-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-02-006.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-01-018.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-07-008.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00254/interview-01-002.wav',
 '/nvme/zhiyong/CN-Celeb

In [26]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[8.,6.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [27]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G9/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G9.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G10

In [28]:
import glob

In [29]:
b = a[180:240]

In [30]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [31]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-005.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-03-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-03-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-02-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-011.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-001.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-013.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-03-001.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-02-003.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-003.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00163/interview-02-001.wav',
 '/nvme/zhiyong/CN-Celeb

In [32]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([4.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    room.add_microphone(loc=[8.,4.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [33]:
glob.glob('/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/*')

['/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00004.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00001.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00006.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00005.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00003.wav',
 '/nvme/zhiyong/voxceleb/vox1/dev/wav/id10143/ioty72DM3MQ/00002.wav']

In [34]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G10/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G10.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G11

In [35]:
import glob

In [36]:
b = a[240:300]

In [37]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [38]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-068.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-104.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-024.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-094.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-052.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-079.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-100.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-056.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-093.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-037.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-03-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-104.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/singing-01-023.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-071.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/interview-02-058.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00666/in

In [39]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))
    # dir_list = []
    # for i in range(6):
    #     dir_list.append(CardioidFamily(
    #     orientation=DirectionVector(azimuth=0, colatitude=90, degrees=True),
    #     pattern_enum=DirectivityPattern.OMNI,))

    # mics = pra.MicrophoneArray(R, room.fs, directivity=dir_list)
    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [40]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G11/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G11.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G12

In [41]:
import glob

In [42]:
b = a[300:360]

In [43]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i+'/*'))

In [44]:
b_list

['/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-006.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-02-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/live_broadcast-01-001.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-01-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-009.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-01-004.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-01-001.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-013.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-02-001.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-017.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-01-003.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-01-002.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/interview-02-014.wav',
 '/nvme/zhiyong/CN-Celeb/data/id00812/entertainment-02-003.w

In [45]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([8.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))

    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [46]:
for j in b_list:
    label = j.split('/')[-2]
    out_path = '/nvme/zhiyong/fllearn_data/G12/'
    generate_wave(j, out_path+label+'_'+j.split('/')[-1])
    with open(out_path+'train_list_G12.txt', 'a') as f:
        f.write(label+' '+out_path+label+'_'+j.split('/')[-1]+'\n')

/tmp/ipykernel_5243/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


# G7 TEST VER

In [16]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [17]:
b = a

In [18]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [19]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [20]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('high', 'small')
    room = pra.ShoeBox(
        p=size_coef*np.array([4, 3, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[3.5,2.5,1.])

    room.simulate()
    
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [21]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G7_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [22]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [23]:
b = a

In [24]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [25]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [26]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G7_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/2301957502.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [1]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [2]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [3]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [4]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G7_VER/'+i.split(' ')[1]+'.wav'+' '
                'G7_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [5]:
ver_new

['1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-001.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-002.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-003.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-004.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-005.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-singing-01-006.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-001.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-004.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-005.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-006.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-007.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-008.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-009.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-011.wav\n',
 '1 G7_VER/id00800-enroll.wav G7_VER/id00800-speech-01-0

In [6]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G7_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G8 TEST VER

In [27]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [28]:
b = a

In [29]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [30]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [31]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('low', 'large')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[17.,12.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [32]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G8_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [33]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [34]:
b = a

In [35]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [36]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [37]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G8_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/287942140.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [7]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [8]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [9]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [11]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G8_VER/'+i.split(' ')[1]+'.wav'+' '
                'G8_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [12]:
ver_new

['1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-001.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-002.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-003.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-004.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-005.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-singing-01-006.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-001.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-004.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-005.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-006.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-007.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-008.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-009.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-011.wav\n',
 '1 G8_VER/id00800-enroll.wav G8_VER/id00800-speech-01-0

In [13]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G8_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G9 TEST VER

In [38]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [39]:
b = a

In [40]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [41]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [42]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    room.add_microphone(loc=[8.,6.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [43]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G9_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [44]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [45]:
b = a

In [46]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [47]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [48]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G9_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/1847509705.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [14]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [15]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [16]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [17]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G9_VER/'+i.split(' ')[1]+'.wav'+' '
                'G9_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [18]:
ver_new

['1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-001.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-002.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-003.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-004.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-005.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-singing-01-006.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-001.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-004.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-005.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-006.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-007.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-008.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-009.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-011.wav\n',
 '1 G9_VER/id00800-enroll.wav G9_VER/id00800-speech-01-0

In [19]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G9_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G10 TEST VER

In [49]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [50]:
b = a

In [51]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [52]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [53]:
def generate_wave(path, out_path):
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([4.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    room.add_microphone(loc=[8.,4.,1.])

    room.simulate()
    to_wav(out_path, fs, room.mic_array.signals[0,:], norm=True, bitdepth=np.int16)

In [54]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G10_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [55]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [56]:
b = a

In [57]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [58]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [59]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G10_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/3125184528.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [20]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [21]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [22]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [23]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G10_VER/'+i.split(' ')[1]+'.wav'+' '
                'G10_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [24]:
ver_new

['1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-001.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-002.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-003.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-004.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-005.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-006.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-001.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-004.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-005.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-006.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-007.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-008.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-009.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-011.wav\n',
 '1 G10_VER/id00800-enroll.w

In [25]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G10_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G11 TEST VER

In [60]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [61]:
b = a

In [62]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [63]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [64]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))
    # dir_list = []
    # for i in range(6):
    #     dir_list.append(CardioidFamily(
    #     orientation=DirectionVector(azimuth=0, colatitude=90, degrees=True),
    #     pattern_enum=DirectivityPattern.OMNI,))

    # mics = pra.MicrophoneArray(R, room.fs, directivity=dir_list)
    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [65]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G11_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [66]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [67]:
b = a

In [68]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [69]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [70]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G11_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/1046023876.py:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [26]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [27]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [28]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [30]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G10_VER/'+i.split(' ')[1]+'.wav'+' '
                'G10_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [31]:
ver_new

['1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-001.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-002.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-003.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-004.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-005.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-singing-01-006.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-001.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-004.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-005.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-006.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-007.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-008.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-009.wav\n',
 '1 G10_VER/id00800-enroll.wav G10_VER/id00800-speech-01-011.wav\n',
 '1 G10_VER/id00800-enroll.w

In [32]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G10_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)

# G12 TEST VER

In [71]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/enroll/*')

In [72]:
b = a

In [73]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [74]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/enroll/id00866-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00801-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00895-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00816-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00912-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00999-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00918-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00991-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00863-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00872-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00970-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00835-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00913-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00857-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00802-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00838-enroll.wav',
 '/nvme/zhiyong/CN-Celeb/eval/enroll/id00811-enroll.wav',
 '/nvme/zhiyon

In [75]:
def generate_wave(path, out_path):
    fft_len = 512
    Lg_t = 0.100                # filter size in seconds
    Lg = np.ceil(Lg_t*16000)       # in samples
    absor, size_coef = room_properties('medium', 'medium')
    room = pra.ShoeBox(
        p=np.array([4*size_coef, 3*size_coef, 2.5]),
        materials=pra.Material(absor),
        fs=16000,
        max_order=3,
        ray_tracing=True,
        air_absorption=True,
    )
    room.set_ray_tracing(receiver_radius=0.1, n_rays=10000, energy_thres=1e-5)

    # specify signal source
    fs, signal = wavfile.read(path)

    # add source to 2D room
    room.add_source([1.,1.,1.], signal=signal)

    # add noise source
    fs, noise = wavfile.read("/nvme/zhiyong/musan/noise/free-sound/noise-free-sound-0000.wav")

    room.add_source([8.,7.,1.], signal=noise[:len(signal)], delay=0.)

    # add microphone array
    R = pra.circular_2D_array(center=[8.,2.], M=6, phi0=0, radius=0.05)
    R = np.concatenate((R, np.ones((1, 6))))

    mics = pra.Beamformer(R, fs, N=fft_len, Lg=Lg)
    room.add_microphone_array(mics)

    # Compute DAS weights
    mics.rake_delay_and_sum_weights(room.sources[0][:1])

    room.simulate()
    signal_das = mics.process(FD=False)
    to_wav(out_path, fs, signal_das, norm=True, bitdepth=np.int16)

In [76]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G12_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]


In [77]:
import glob
a = glob.glob('/nvme/zhiyong/CN-Celeb/eval/test/*')

In [78]:
b = a

In [79]:
b_list = []
for i in b:
    b_list.extend(glob.glob(i))

In [80]:
b_list

['/nvme/zhiyong/CN-Celeb/eval/test/id00927-live_broadcast-01-110.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00854-entertainment-03-012.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00972-singing-02-028.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00850-interview-02-010.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00829-entertainment-04-002.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00887-speech-02-007.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00932-entertainment-03-005.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00847-singing-01-132.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00952-entertainment-08-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00852-singing-05-013.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00911-interview-02-081.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00893-singing-03-003.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00917-interview-01-069.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00930-interview-01-006.wav',
 '/nvme/zhiyong/CN-Celeb/eval/test/id00904-singing-01-001.wav',
 '

In [81]:
for j in b_list:
    out_path = '/nvme/zhiyong/fllearn_data/G12_VER/'
    generate_wave(j, out_path+j.split('/')[-1])

/tmp/ipykernel_42082/3403348603.py:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  float_types = [float, np.float, np.float32, np.float64]
/tmp/ipykernel_42082/1548611480.py:17: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, signal = wavfile.read(path)


In [39]:
ver_ori_path = '/nvme/zhiyong/CN-Celeb/CNCELEB_trials_clean.txt'

In [40]:
with open(ver_ori_path, 'r') as f:
    ver_ori = f.readlines()

In [41]:
ver_ori

['1 id00800-enroll eval/test/id00800-singing-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-002.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-003.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-singing-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-001.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-004.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-005.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-006.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-007.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-008.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-009.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-011.wav\n',
 '1 id00800-enroll eval/test/id00800-speech-01-013.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-01-001.wav\n',
 '0 id00800-enroll eval/test/id00801-drama-02-001.w

In [42]:
ver_new = []
for i in ver_ori:
    i = i.strip()
    ver_new.append(i.split(' ')[0]+' '
                'G11_VER/'+i.split(' ')[1]+'.wav'+' '
                'G11_VER/'+i.split(' ')[2].split('/')[-1]+'\n')

In [43]:
ver_new

['1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-001.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-002.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-003.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-004.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-005.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-singing-01-006.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-001.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-004.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-005.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-006.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-007.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-008.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-009.wav\n',
 '1 G11_VER/id00800-enroll.wav G11_VER/id00800-speech-01-011.wav\n',
 '1 G11_VER/id00800-enroll.w

In [44]:
# write ver_new to .txt file
with open('/nvme/zhiyong/fllearn_data/ver_list_G11_VER.txt', 'w') as f:
    for i in ver_new:
        f.write(i)